In [1689]:
import pandas as pd
import os

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from spacy import displacy
import edsnlp, edsnlp.pipes as eds

input_dir = "../data_clean/"
visuals_dir = "../visuals/" 

In [1690]:
df_person = pd.read_pickle(os.path.join(input_dir, 'df_person.pkl'))
df_bio = pd.read_pickle(os.path.join(input_dir, 'df_bio.pkl'))
df_note = pd.read_pickle(os.path.join(input_dir, 'df_note.pkl'))
df_visit = pd.read_pickle(os.path.join(input_dir, 'df_visit.pkl'))
df_condition = pd.read_pickle(os.path.join(input_dir, 'df_condition.pkl'))

In [1691]:
nlp = edsnlp.blank("eds")
nlp.add_pipe(eds.sentences())
nlp.add_pipe(eds.normalizer())
nlp.add_pipe(eds.tobacco())
nlp.add_pipe(eds.negation())
nlp.add_pipe(eds.family())

smokers = 0
total = 0
for i in range(len(df_note)):
    note = df_note.iloc[i]
    text = note['note_text']
    doc = nlp(text)

    filtered_ents_for_display = [ent for ent in doc.ents if not ent._.family]
    filtered_ents_for_display = [ent for ent in filtered_ents_for_display if not ent._.negation]

    visit_id_from_sampled_note = note['visit_occurrence_id']
    matching_visit_rows = df_visit[df_visit['visit_occurrence_id'] == visit_id_from_sampled_note]

    current_note_id_scalar = note['note_id']
    person_id = matching_visit_rows['person_id'].iloc[0]
   
    if matching_visit_rows.empty:
        raise ValueError(f"No matching visit found for visit_occurrence_id: {visit_id_from_sampled_note}")
    total += 1
    if filtered_ents_for_display: 
        smokers += 1
        df_person.loc[df_person['person_id'] == person_id, 'fumeur'] = True
    else:
        df_person.loc[df_person['person_id'] == person_id, 'fumeur'] = False

print(f"Total notes processed: {total}")
print(f"Total smokers identified: {smokers}")

2025-06-04 15:09:07.328 | WARNING  | edsnlp.pipes.qualifiers.family.family:__init__:171 - You have requested that the pipeline use annotations provided by the `section` pipeline, but it was not set. Skipping that step.


Total notes processed: 958
Total smokers identified: 90


In [ ]:
nlp = edsnlp.blank("eds")
nlp.add_pipe(eds.sentences())
nlp.add_pipe(eds.normalizer())
nlp.add_pipe(eds.alcohol())
nlp.add_pipe(eds.negation())
nlp.add_pipe(eds.family())

alcohol = 0
total = 0
for i in range(len(df_note)):
    note = df_note.iloc[i]
    text = note['note_text']
    doc = nlp(text)

    filtered_ents_for_display = [ent for ent in doc.ents if not ent._.family]
    filtered_ents_for_display = [ent for ent in filtered_ents_for_display if not ent._.negation]

    visit_id_from_sampled_note = note['visit_occurrence_id']
    matching_visit_rows = df_visit[df_visit['visit_occurrence_id'] == visit_id_from_sampled_note]

    current_note_id_scalar = note['note_id']
    person_id = matching_visit_rows['person_id'].iloc[0]
   
    if matching_visit_rows.empty:
        raise ValueError(f"No matching visit found for visit_occurrence_id: {visit_id_from_sampled_note}")
    total += 1
    if filtered_ents_for_display:
        alcohol += 1
        df_person.loc[df_person['person_id'] == person_id, 'alcool'] = True
    else:
        df_person.loc[df_person['person_id'] == person_id, 'alcool'] = False
        
print(f"Total notes processed: {total}")
print(f"Total alcohol consumers identified: {alcohol}")

2025-06-04 15:09:27.345 | WARNING  | edsnlp.pipes.qualifiers.family.family:__init__:171 - You have requested that the pipeline use annotations provided by the `section` pipeline, but it was not set. Skipping that step.


Total notes processed: 958
Total alcohol consumers identified: 0
